### Notebook for concept detection in neural network

In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
session_name = "starship"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 150, 300, 450, 600, 750, 900, 1050, 1200, 1500, 1800]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = concepts.play_center_in_opening

CONCEPT_NAME = concepts.play_center_in_opening.__name__

CASES_TO_SAMPLE = 1000 # 25000


In [14]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [18]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0
    random_moves = False

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.2
    alpha = 0.2 # The higher, the more unique games. 0 = argmax, 1 = distribution 

    game_over = False

    while not game_over:
        if moves == total_moves:
            break
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            pos = concept_function(state)
            if pos:
                positive_cases.append(state_copy)
            elif not pos:
                negative_cases.append(state_copy)

        if random_moves:
            action = go_env.uniform_random_action()
        else:
            action = agents[current_player].best_action(state, alpha=alpha)

        state, _, game_over, _ = go_env.step(action)
        moves += 1

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE or len(negative_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)

            positive_cases.extend(pos)
            negative_cases.extend(neg)

            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases:   6%|▌         | 60/1000 [00:13<03:36,  4.35it/s]
Positive cases: 1033it [03:53,  6.55it/s]                         

In [19]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (1000, 4, 5, 5)
Negative cases:  (1000, 4, 5, 5)


In [20]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [21]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  633
Duplicate count in negative:  39


In [22]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

Positive case 0:
[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]]

Positive case 1:
[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]]

Positive case 2:
[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.

In [88]:
# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

BINARY = True

Positions to consider: 1600


In [87]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = concepts.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Regression score:  0.5785729194955613


In [89]:
epochs_to_look_at = [0, 150, 300, 450, 600, 750, 900, 1050, 1200, 1500, 1800]

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = concepts.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)


Getting activation outputs: 100%|██████████| 63/63 [00:00<00:00, 268.63it/s]


Performing regression for layer 0
Epoch 1/50
50/50 [==============================] - 0s 3ms/step - loss: 0.9580 - val_loss: 0.9435
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9308 - val_loss: 0.9168
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9053 - val_loss: 0.8921
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8817 - val_loss: 0.8693
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 0.8600 - val_loss: 0.8481
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8398 - val_loss: 0.8284
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8209 - val_loss: 0.8097
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8032 - val_loss: 0.7926
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7868 - val_loss: 0.7767
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7717 - va

Getting activation outputs: 100%|██████████| 63/63 [00:00<00:00, 309.63it/s]

Performing regression for layer 0
Epoch 1/50


50/50 [==============================] - 1s 3ms/step - loss: 0.9498 - val_loss: 0.9327
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8929 - val_loss: 0.8946
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8511 - val_loss: 0.8609
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8166 - val_loss: 0.8326
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7868 - val_loss: 0.8072
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7608 - val_loss: 0.7840
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7377 - val_loss: 0.7635
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7169 - val_loss: 0.7435
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6982 - val_loss: 0.7256
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6803 - val_loss: 0.7093
Epoch 11/50
50/50 [===========

...

Performing regression for layer 0
Epoch 1/50


50/50 [==============================] - 0s 3ms/step - loss: 1.0407 - val_loss: 0.9851
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9498 - val_loss: 0.9105
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8790 - val_loss: 0.8499
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8220 - val_loss: 0.8020
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7759 - val_loss: 0.7632
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7392 - val_loss: 0.7317
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7081 - val_loss: 0.7053
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6821 - val_loss: 0.6822
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6599 - val_loss: 0.6627
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6404 - val_loss: 0.6455
Epoch 11/50
50/50 [===========

Getting activation outputs: 100%|██████████| 63/63 [00:00<00:00, 294.33it/s]

Performing regression for layer 0
Epoch 1/50


50/50 [==============================] - 0s 3ms/step - loss: 1.0622 - val_loss: 0.9739
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9625 - val_loss: 0.9060
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8862 - val_loss: 0.8512
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8268 - val_loss: 0.8074
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7798 - val_loss: 0.7720
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7425 - val_loss: 0.7428
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7111 - val_loss: 0.7178
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6848 - val_loss: 0.6970
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6626 - val_loss: 0.6784
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.6432 - val_loss: 0.6622
Epoch 11/50
50/50 [===========